огромная благодарность  [WILMER E. HENAO](https://www.kaggle.com/verracodeguacas)


🚀 Введение 🌟



В огромном мире машинного обучения многие из нас знакомы с обычными подходами: LightGBM, XGBoost, CatBoost и различными нейронными сетями, такими как LSTM. Однако в этой записной книжке мы пойдем другим путем. 🔄 Мы погружаемся в мир регрессий на опорных векторах (SVR). 🌐

🔍 Что такое SVR? 🤔

SVR, член семейства машин опорных векторов (SVM), по сути, является методом регрессии. Проще говоря, если классический SVM используется для задач классификации, то SVR применяется для задач регрессии. Представьте себе SVM как умный алгоритм, который находит лучшую линию (или гиперплоскость в более высоких измерениях) для разделения двух классов. SVR, с другой стороны, использует похожий подход для подгонки лучшей линии в пределах заданного порога ошибки для набора точек данных. Это делает его особенно полезным для прогнозирования и предсказания непрерывных значений. 📈

Что отличает SVR, так это его способность управлять нелинейными отношениями с помощью ядер. Я утверждаю, что хорошо реализованный SVR с RBF-ядром может конкурировать с инструментами градиентного бустинга в любой день! Ядра преобразуют данные в более высокое измерение, где возможно найти линейное разделение (или максимально приближенное к нему). 🌐⚙️

🔄 SVR против SVC: Связь и Различие 🔄

В то время как SVR сосредоточен на регрессии, его "родственник", классификация на опорных векторах (SVC), занимается задачами классификации. Оба они имеют общую основу максимизации зазора и минимизации ошибки классификации/регрессии, но отличаются в своих конечных целях. SVC стремится найти лучшую разделяющую гиперплоскость для классификации точек данных, в то время как SVR пытается подогнать лучшую линию в определенном допуске к точкам данных. Несмотря на эти различия, их основные принципы и математика тесно связаны. 🤝

⏱️ Вызов Калибровки SVR ⏱️

Важный момент, который следует отметить о SVR, это время его калибровки. SVR может быть медленным в настройке и калибровке из-за его сложности, особенно при работе с большими наборами данных. Поэтому мудро провести этот этап калибровки заранее и сохранить откалиброванную модель для будущего использования. 🔄💾

📚 Путешествие по этой записной книжке 📚

В этой записной книжке мы начнем с руководства по оффлайн-установке RAPIDS, набора библиотек с открытым исходным кодом для выполнения полных циклов науки о данных исключительно на GPU, что значительно ускоряет процесс науки о данных. 🚀 Я проведу установку с помощью вспомогательной записной книжки: https://www.kaggle.com/code/verracodeguacas/rapids-installer, не стесняйтесь использовать эту записную книжку, чтобы узнать, как устанавливать программы оффлайн для этого и других соревнований Kaggle.

Далее мы перейдем к обучению модели SVR. Хотя я закомментировал фактический код обучения, он там для того, чтобы вы могли экспериментировать и калибровать свои модели. Калибровка может занять много времени, поэтому я предоставил ссылку на предварительно обученную модель SVR: Набор данных моделей SVR https://www.kaggle.com/datasets/verracodeguacas/svr-models. 📈💻

Наконец, мы рассмотрим, как использовать эту обученную модель SVR для предсказаний и продемонстрируем, как ансамблировать ее с другими популярными моделями, такими как LightGBM. Этот подход позволяет нам использовать сильные стороны различных моделей для повышения производительности. 🤖💥

Приготовьтесь к захватывающему путешествию в мир SVR, менее изученному пути с большим потенциалом в машинном обучении! 🌟


## RAPIDS offline installation

In [ ]:
# Удаляем библиотеки cupy и cupy-cuda11x, если они установлены
!pip uninstall cupy, cupy-cuda11x

# Устанавливаем пакеты cudf-cu11 и cuml-cu11 без использования сетевого индекса,
# поиск пакетов осуществляется в локальной директории /kaggle/input/rapids-installer
!pip install --no-index --find-links /kaggle/input/rapids-installer cudf-cu11 cuml-cu11


## игнорируйте ошибки 

In [ ]:
import cuml

In [ ]:
cuml.__version__

SVR,LGBM  библиотеки

In [ ]:
import gc  # Управление сборкой мусора для управления памятью
import os  # Функции, связанные с операционной системой
import time  # Функции, связанные со временем
import warnings  # Обработка предупреждений
from itertools import combinations  # Для создания комбинаций элементов
from warnings import simplefilter  # Упрощение обработки предупреждений

# 📦 Импорт библиотек машинного обучения
import joblib  # Для сохранения и загрузки моделей
import numpy as np  # Нумерические операции
import pandas as pd  # Манипуляции с данными и анализ
from sklearn.metrics import mean_absolute_error  # Метрика для оценки
from sklearn.model_selection import KFold, TimeSeriesSplit  # Техники кросс-валидации
from sklearn.impute import SimpleImputer

# import polars as pl  # Замена pandas на Polars (строка закомментирована)
from cuml.svm import SVR  # Импорт SVR из библиотеки CUML

# 🤐 Отключение предупреждений для чистоты кода
warnings.filterwarnings("ignore")
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

# 📊 Определение флагов и переменных
is_offline = False  # Флаг для режима онлайн/оффлайн
is_train = True  # Флаг для режима обучения
is_infer = True  # Флаг для режима вывода
max_lookback = np.nan  # Максимальный период ретроспективы (не указан)
split_day = 435  # День разделения для временных рядов

import lightgbm as lgb

from numba import njit, prange



In [ ]:
import os
from cuml.svm import SVR
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
import joblib


### Загрузим

In [ ]:


# Чтение данных из CSV-файла, расположенного по указанному пути
df = pd.read_csv("/kaggle/input/optiver-trading-at-the-close/train.csv")

# Удаление строк, в которых отсутствуют данные в столбце "target"
df = df.dropna(subset=["target"])

# Сброс индекса DataFrame для обеспечения последовательной нумерации строк
# Параметр drop=True гарантирует, что старые индексы не будут добавлены как новый столбец
df.reset_index(drop=True, inplace=True)

# Получение размерности DataFrame (количество строк и столбцов)
df_shape = df.shape


In [ ]:
df.head()

Функция reduce_mem_usage предназначена для оптимизации использования памяти DataFrame в Pandas за счет преобразования типов данных столбцов. 

In [ ]:

def reduce_mem_usage(df, verbose=0):
    # Начальный размер использования памяти DataFrame
    start_mem = df.memory_usage().sum() / 1024**2

    # Перебор всех столбцов DataFrame
    for col in df.columns:
        # Получение типа данных столбца
        col_type = df[col].dtype

        # Обработка столбцов, не содержащих объектные типы данных
        if col_type != object:
            # Нахождение минимального и максимального значения в столбце
            c_min = df[col].min()
            c_max = df[col].max()
            
            # Оптимизация типов данных для целочисленных столбцов
            if str(col_type)[:3] == "int":
                # Преобразование типа данных столбца в соответствии с диапазоном значений
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                # Оптимизация типов данных для вещественных чисел
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float32)

    # Вывод информации о памяти, если установлен флаг verbose
    if verbose:
        logger.info(f"Memory usage of dataframe is {start_mem:.2f} MB")
        end_mem = df.memory_usage().sum() / 1024**2
        logger.info(f"Memory usage after optimization is: {end_mem:.2f} MB")
        decrease = 100 * (start_mem - end_mem) / start_mem
        logger.info(f"Decreased by {decrease:.2f}%")

    # Возвращение оптимизированного DataFrame
    return df



две функции, оптимизированные с помощью библиотеки Numba для ускорения вычислений.

функции предназначены для вычисления сложных статистических характеристик данных, таких как дисбаланс троек значений и скользящие средние, с использованием параллельных вычислений для ускорения процесса. Они могут быть особенно полезны в задачах обработки больших объемов данных или временных рядов.

In [ ]:

@njit(parallel=True)
def compute_triplet_imbalance(df_values, comb_indices):
    """
    Вычисляет дисбаланс троек значений для каждой строки в массиве.

    Параметры:
    df_values (numpy.ndarray): 2D массив значений.
    comb_indices (List[Tuple[int, int, int]]): Список индексов для комбинаций троек столбцов.

    Возвращает:
    numpy.ndarray: 2D массив с вычисленными значениями дисбаланса.
    """
    num_rows = df_values.shape[0]
    num_combinations = len(comb_indices)
    imbalance_features = np.empty((num_rows, num_combinations))

    for i in prange(num_combinations):
        a, b, c = comb_indices[i]
        for j in range(num_rows):
            max_val = max(df_values[j, a], df_values[j, b], df_values[j, c])
            min_val = min(df_values[j, a], df_values[j, b], df_values[j, c])
            mid_val = df_values[j, a] + df_values[j, b] + df_values[j, c] - min_val - max_val
            
            if mid_val == min_val:
                imbalance_features[j, i] = np.nan
            else:
                imbalance_features[j, i] = (max_val - mid_val) / (mid_val - min_val)

    return imbalance_features

def calculate_triplet_imbalance_numba(price, df):
    """
    Обертка для compute_triplet_imbalance, преобразующая DataFrame в массив и создающая комбинации.

    Параметры:
    price (List[str]): Список названий столбцов для комбинаций.
    df (pandas.DataFrame): DataFrame с данными.

    Возвращает:
    pandas.DataFrame: DataFrame с вычисленными значениями дисбаланса.
    """
    df_values = df[price].values
    comb_indices = [(price.index(a), price.index(b), price.index(c)) for a, b, c in combinations(price, 3)]
    features_array = compute_triplet_imbalance(df_values, comb_indices)
    columns = [f"{a}_{b}_{c}_imb2" for a, b, c in combinations(price, 3)]
    features = pd.DataFrame(features_array, columns=columns)
    return features

@njit(fastmath=True)
def rolling_average(arr, window):
    """
    Вычисляет скользящее среднее для одномерного массива numpy.

    Параметры:
    arr (numpy.ndarray): Входной массив для вычисления скользящего среднего.
    window (int): Количество элементов для учета в скользящем среднем.

    Возвращает:
    numpy.ndarray: Массив со значениями скользящего среднего.
    """
    n = len(arr)
    result = np.empty(n)
    result[:window] = np.nan  # Заполнение NaN для элементов, где окно не полное
    cumsum = np.cumsum(arr)

    for i in range(window, n):
        result[i] = (cumsum[i] - cumsum[i - window]) / window

    return result

@njit(parallel=True)
def compute_rolling_averages(df_values, window_sizes):
    """
    Вычисляет скользящие средние для нескольких размеров окон параллельно.

    Параметры:
    df_values (numpy.ndarray): 2D массив значений для вычисления скользящих средних.
    window_sizes (List[int]): Список размеров окон для скользящих средних.

    Возвращает:
    numpy.ndarray: 3D массив со скользящими средними для каждого размера окна.
    """
    num_rows, num_features = df_values.shape
    num_windows = len(window_sizes)
    rolling_features = np.empty((num_rows, num_features, num_windows))

    for feature_idx in prange(num_features):
        for window_idx, window in enumerate(window_sizes):
            rolling_features[:, feature_idx, window_idx] = rolling_average(df_values[:, feature_idx], window)

    return rolling_features


три функции, которые генерируют различные SVR  признаки для DataFrame. Каждая функция выполняет свою уникальную задачу по обработке и созданию новых признаков.

In [ ]:


def imbalance_features(df):
    """
    Генерирует признаки, связанные с дисбалансом цен и размеров в DataFrame.

    Параметры:
    df (pandas.DataFrame): Исходный DataFrame.

    Возвращает:
    pandas.DataFrame: DataFrame с добавленными признаками дисбаланса.
    """
    # Определение списков названий столбцов, связанных с ценами и размерами
    prices = ["reference_price", "far_price", "near_price", "ask_price", "bid_price", "wap"]
    sizes = ["matched_size", "bid_size", "ask_size", "imbalance_size"]

    # Вычисление новых признаков
    df["volume"] = df.eval("ask_size + bid_size")
    df["mid_price"] = df.eval("(ask_price + bid_price) / 2")
    df["liquidity_imbalance"] = df.eval("(bid_size-ask_size)/(bid_size+ask_size)")
    df["matched_imbalance"] = df.eval("(imbalance_size-matched_size)/(matched_size+imbalance_size)")
    df["size_imbalance"] = df.eval("bid_size / ask_size")

    # Вычисление дисбаланса между парами цен
    for c in combinations(prices, 2):
        df[f"{c[0]}_{c[1]}_imb"] = df.eval(f"({c[0]} - {c[1]})/({c[0]} + {c[1]})")

    # Вычисление дисбаланса троек значений
    for c in [['ask_price', 'bid_price', 'wap', 'reference_price'], sizes]:
        triplet_feature = calculate_triplet_imbalance_numba(c, df)
        df[triplet_feature.columns] = triplet_feature.values
   
    # Вычисление различных динамических признаков
    df["imbalance_momentum"] = df.groupby(['stock_id'])['imbalance_size'].diff(periods=1) / df['matched_size']
    df["price_spread"] = df["ask_price"] - df["bid_price"]
    df["spread_intensity"] = df.groupby(['stock_id'])['price_spread'].diff()
    df['price_pressure'] = df['imbalance_size'] * (df['ask_price'] - df['bid_price'])
    df['market_urgency'] = df['price_spread'] * df['liquidity_imbalance']
    df['depth_pressure'] = (df['ask_size'] - df['bid_size']) * (df['far_price'] - df['near_price'])
    
    # Вычисление статистических агрегаций
    for func in ["mean", "std", "skew", "kurt"]:
        df[f"all_prices_{func}"] = df[prices].agg(func, axis=1)
        df[f"all_sizes_{func}"] = df[sizes].agg(func, axis=1)
        
    # Вычисление сдвигов и относительных изменений для определенных столбцов
    for col in ['matched_size', 'imbalance_size', 'reference_price', 'imbalance_buy_sell_flag']:
        for window in [1, 2, 3, 10]:
            df[f"{col}_shift_{window}"] = df.groupby('stock_id')[col].shift(window)
            df[f"{col}_ret_{window}"] = df.groupby('stock_id')[col].pct_change(window)
    
    # Вычисление разностных признаков
    for col in ['ask_price', 'bid_price', 'ask_size', 'bid_size', 'market_urgency', 'imbalance_momentum', 'size_imbalance']:
        for window in [1, 2, 3, 10]:
            df[f"{col}_diff_{window}"] = df.groupby("stock_id")[col].diff(window)

    return df.replace([np.inf, -np.inf], 0)

def other_features(df):
    """
    Генерирует дополнительные признаки, не связанные с дисбалансом.

    Параметры:
    df (pandas.DataFrame): DataFrame для добавления признаков.

    Возвращает:
    pandas.DataFrame: DataFrame с добавленными признаками.
    """
    # Вычисление дня недели, секунд и минут
    df["dow"] = df["date_id"] % 5  # День недели
    df["seconds"] = df["seconds_in_bucket"] % 60  
    df["minute"] = df["seconds_in_bucket"] // 60  

    # Применение глобальных признаков к stock_id
    for key, value in global_stock_id_feats.items():
        df[f"global_{key}"] = df["stock_id"].map(value.to_dict())

    return df

def generate_all_features(df):
    """
    Генерирует все признаки для DataFrame.

    Параметры:
    df (pandas.DataFrame): Исходный DataFrame.

    Возвращает:
    pandas.DataFrame: DataFrame со всеми сгенерированными признаками.
    """
    # Выбор релевантных столбцов для генерации признаков
    cols = [c for c in df.columns if c not in ["row_id", "time_id", "target"]]
    df = df[cols]
    
    # Генерация признаков дисбаланса и других признаков
    df = imbalance_features(df)
    df = other_features(df)
    gc.collect()  # Очистка памяти
    feature_name = [i for i in df.columns if i not in ["row_id", "target", "time_id", "date_id"]]
    
    return df[feature_name]


### веса для загружаемых моделей LGB

In [ ]:
weights = [
    0.004, 0.001, 0.002, 0.006, 0.004, 0.004, 0.002, 0.006, 0.006, 0.002, 0.002, 0.008,
    0.006, 0.002, 0.008, 0.006, 0.002, 0.006, 0.004, 0.002, 0.004, 0.001, 0.006, 0.004,
    0.002, 0.002, 0.004, 0.002, 0.004, 0.004, 0.001, 0.001, 0.002, 0.002, 0.006, 0.004,
    0.004, 0.004, 0.006, 0.002, 0.002, 0.04 , 0.002, 0.002, 0.004, 0.04 , 0.002, 0.001,
    0.006, 0.004, 0.004, 0.006, 0.001, 0.004, 0.004, 0.002, 0.006, 0.004, 0.006, 0.004,
    0.006, 0.004, 0.002, 0.001, 0.002, 0.004, 0.002, 0.008, 0.004, 0.004, 0.002, 0.004,
    0.006, 0.002, 0.004, 0.004, 0.002, 0.004, 0.004, 0.004, 0.001, 0.002, 0.002, 0.008,
    0.02 , 0.004, 0.006, 0.002, 0.02 , 0.002, 0.002, 0.006, 0.004, 0.002, 0.001, 0.02,
    0.006, 0.001, 0.002, 0.004, 0.001, 0.002, 0.006, 0.006, 0.004, 0.006, 0.001, 0.002,
    0.004, 0.006, 0.006, 0.001, 0.04 , 0.006, 0.002, 0.004, 0.002, 0.002, 0.006, 0.002,
    0.002, 0.004, 0.006, 0.006, 0.002, 0.002, 0.008, 0.006, 0.004, 0.002, 0.006, 0.002,
    0.004, 0.006, 0.002, 0.004, 0.001, 0.004, 0.002, 0.004, 0.008, 0.006, 0.008, 0.002,
    0.004, 0.002, 0.001, 0.004, 0.004, 0.004, 0.006, 0.008, 0.004, 0.001, 0.001, 0.002,
    0.006, 0.004, 0.001, 0.002, 0.006, 0.004, 0.006, 0.008, 0.002, 0.002, 0.004, 0.002,
    0.04 , 0.002, 0.002, 0.004, 0.002, 0.002, 0.006, 0.02 , 0.004, 0.002, 0.006, 0.02,
    0.001, 0.002, 0.006, 0.004, 0.006, 0.004, 0.004, 0.004, 0.004, 0.002, 0.004, 0.04,
    0.002, 0.008, 0.002, 0.004, 0.001, 0.004, 0.006, 0.004,
]
weights = {int(k):v for k,v in enumerate(weights)}

код разделяет данные на обучающий и валидационный наборы в зависимости от того, находится ли процесс в оффлайн-режиме или онлайн-режиме.

In [ ]:
# Проверка, находимся ли мы в оффлайн-режиме
if is_offline:
    # Если в оффлайн-режиме, разделяем данные на обучающий и валидационный наборы
    # на основе указанного дня разделения (split_day)
    df_train = df[df["date_id"] <= split_day]
    df_valid = df[df["date_id"] > split_day]
    
    # Вывод информации о режиме и размерах наборов данных
    print("Offline mode")
    print(f"train : {df_train.shape}, valid : {df_valid.shape}")
else:
    # Если в онлайн-режиме, используем весь набор данных для обучения
    df_train = df
    print("Online mode")


### Этот код создаёт набор признаков для обучающего и валидационного наборов данных, если активирован режим обучения (is_train). SVR

In [ ]:
# Проверка, активирован ли режим обучения
if is_train:
    # Создание глобальных признаков для каждого stock_id
    global_stock_id_feats = {
        "median_size": df_train.groupby("stock_id")["bid_size"].median() + df_train.groupby("stock_id")["ask_size"].median(),
        "std_size": df_train.groupby("stock_id")["bid_size"].std() + df_train.groupby("stock_id")["ask_size"].std(),
        "ptp_size": df_train.groupby("stock_id")["bid_size"].max() - df_train.groupby("stock_id")["bid_size"].min(),
        "median_price": df_train.groupby("stock_id")["bid_price"].median() + df_train.groupby("stock_id")["ask_price"].median(),
        "std_price": df_train.groupby("stock_id")["bid_price"].std() + df_train.groupby("stock_id")["ask_price"].std(),
        "ptp_price": df_train.groupby("stock_id")["bid_price"].max() - df_train.groupby("stock_id")["ask_price"].min(),
    }

    # Если в оффлайн-режиме, генерируем признаки для обучающего и валидационного наборов
    if is_offline:
        df_train_feats = generate_all_features(df_train)
        print("Build Train Feats Finished.")
        df_valid_feats = generate_all_features(df_valid)
        print("Build Valid Feats Finished.")
        df_valid_feats = reduce_mem_usage(df_valid_feats)
    else:
        # Если в онлайн-режиме, генерируем признаки только для обучающего набора
        df_train_feats = generate_all_features(df_train)
        print("Build Online Train Feats Finished.")

    # Оптимизация использования памяти для обучающего набора признаков
    df_train_feats = reduce_mem_usage(df_train_feats)


### В комментариях ниже вы найдете тренировочные циклы SVR. Каждое сгибание занимает около 10 часов. Тренировка занимает 10 часов, так что наберитесь терпения

закоментирован, сделал разбиение по фолдам и каждая модель на новом фолде обучается 
время обучения

In [ ]:


# # Инициализация импутера для замены отсутствующих значений средними
# imputer = SimpleImputer(strategy='mean')

# # Задание параметров для модели SVR
# svr_params = {
#     "kernel": "rbf",  # Тип ядра
#     "C": 1,           # Параметр регуляризации
#     "gamma": 'scale', # Коэффициент для ядра
# }

# # Получение списка названий признаков
# feature_name = list(df_train_feats.columns)
# print(f"Длина списка признаков = {len(feature_name)}")

# # Настройка параметров кросс-валидации
# num_folds = 5
# fold_size = 480 // num_folds
# gap = 5

# # Папка для сохранения данных фолдов
# fold_data_save_path = 'fold_data'
# if not os.path.exists(fold_data_save_path):
#     os.makedirs(fold_data_save_path)

# # Получение идентификаторов дат
# date_ids = df_train['date_id'].values

# for i in range(num_folds):
#     start = i * fold_size
#     end = start + fold_size
    
#     # Определение индексов для обучающего и валидационного наборов
#     if i < num_folds - 1:
#         purged_start = end - 2
#         purged_end = end + gap + 2
#         train_indices = (date_ids >= start) & (date_ids < purged_start) | (date_ids > purged_end)
#     else:
#         train_indices = (date_ids >= start) & (date_ids < end)
    
#     test_indices = (date_ids >= end) & (date_ids < end + fold_size)
    
#     # Создание обучающих и валидационных наборов данных
#     df_fold_train = df_train_feats[train_indices]
#     df_fold_train_target = df_train['target'][train_indices]
#     df_fold_valid = df_train_feats[test_indices]
#     df_fold_valid_target = df_train['target'][test_indices]
#     print(f"Фолд {i+1}: Размер тренировочного набора = {df_fold_train.shape}, Размер валидационного набора = {df_fold_valid.shape}")

#     # Сохранение данных фолда в файл
#     fold_data = {
#         'train': df_fold_train,
#         'train_target': df_fold_train_target,
#         'valid': df_fold_valid,
#         'valid_target': df_fold_valid_target
#     }
#     joblib.dump(fold_data, os.path.join(fold_data_save_path, f'fold_data_{i+1}.joblib'))

#     print(f"Данные фолда {i+1} сохранены")

# print("Все фолды подготовлены и сохранены")


In [ ]:
# import os
# from cuml.svm import SVR
# from sklearn.metrics import mean_absolute_error
# from sklearn.impute import SimpleImputer
# import joblib
# from cuml.common.logger import set_level, level_info  # Импорт для логирования

# # Установка уровня логирования
# set_level(level_info)

In [ ]:
# import os
# from cuml.svm import SVR
# from sklearn.metrics import mean_absolute_error
# from sklearn.impute import SimpleImputer
# import joblib

# # Установите номер фолда вручную здесь перед каждым запуском
# fold_number = 1  # Номер фолда (1, 2, 3, ...)

# # Загрузка данных фолда
# fold_data_path = os.path.join('fold_data', f'fold_data_{fold_number}.joblib')
# fold_data = joblib.load(fold_data_path)

# # Нормализация данных
# scaler = MinMaxScaler((0, 1))
# df_fold_train_scaled = scaler.fit_transform(fold_data['train'])
# df_fold_valid_scaled = scaler.transform(fold_data['valid'])

# # Импутация отсутствующих значений
# imputer = SimpleImputer(strategy='mean')
# df_fold_train_imputed = imputer.fit_transform(df_fold_train_scaled)
# df_fold_valid_imputed = imputer.transform(df_fold_valid_scaled)

# # # Импутация отсутствующих значений
# # imputer = SimpleImputer(strategy='mean')
# # df_fold_train_imputed = imputer.fit_transform(fold_data['train'])
# # df_fold_valid_imputed = imputer.transform(fold_data['valid'])

# # Параметры модели SVR
# svr_params = {
#     "kernel": "rbf",
#     "C": 1,
#     "gamma": 'scale'
#     #"max_iter": 40000  # Ограничение количества итераций
# }

# # # Обучение модели SVR
# # svr_model = SVR(**svr_params)
# # svr_model.fit(df_fold_train_imputed, fold_data['train_target'])

# # Обучение модели SVR
# svr_model = SVR(**svr_params, verbose=True)  # Добавлен параметр verbose
# svr_model.fit(df_fold_train_imputed, fold_data['train_target'])


# # Папка для сохранения моделей
# model_save_path = 'modelitos_para_despues'
# if not os.path.exists(model_save_path):
#     os.makedirs(model_save_path)

# # Сохранение обученной модели
# model_filename = os.path.join(model_save_path, f'svr_model_fold_{fold_number}.joblib')
# joblib.dump(svr_model, model_filename)

# # Оценка производительности
# fold_predictions = svr_model.predict(df_fold_valid_imputed)
# fold_score = mean_absolute_error(fold_data['valid_target'], fold_predictions)

# print(f"Модель для фолда {fold_number} сохранена в {model_filename}")
# print(f"MAE для фолда {fold_number}: {fold_score}")


In [ ]:
# # Обучение финальной модели на всем наборе данных
# final_model = SVR(**svr_params)
# final_model.fit(df_train_feats[feature_name], df_train['target'])
# 12zq
# # Сохранение финальной модели
# # final_model_filename = os.path.join(model_save_path, 'final_svr_model.joblib')
# final_model_filename = os.path.join(model_save_path, '/kaggle/input/svm-kagle/svr_model_fold_1.joblib')
# joblib.dump(final_model, final_model_filename)

# print(f"Финальная модель сохранена в {final_model_filename}")
# print(f"Средняя MAE по всем фолдам: {np.mean(scores)}")

## загрузка моделей

In [ ]:

# # загрузка ранее посчитаной модели
# final_model = joblib.load("/kaggle/input/svm-kagle/svr_model_fold_1.joblib")

In [ ]:
# models = [final_model]

In [ ]:
# models

In [ ]:
# import joblib
# import numpy as np

# # Загрузка моделей
# models = []
# for i in range(1, 6):
#     model = joblib.load(f"/kaggle/input/svm-kagle/svr_model_fold_{i}.joblib")
#     models.append(model)

# # Предположим, у тебя есть данные для предсказания X_test
# # Метод для усреднения предсказаний
# def average_predictions(models, X_test):
#     predictions = [model.predict(X_test) for model in models]
#     return np.mean(predictions, axis=0)

# # Получение финальных предсказаний
# final_predictions = average_predictions(models, X_test)

# Теперь final_predictions - это усредненные предсказания от всех твоих моделей


## Загрузка других моделей, которые вы можете использовать для создания ансамбля


загружает предобученные модели LightGBM из указанных папок.
Каждая папка содержит модели для разных фолдов кросс-валидации и одну финальную модель

In [ ]:

def load_models_from_folder(model_save_path, num_folds=5):
    """
    Функция для загрузки моделей LightGBM из заданной директории. 
    Модели предполагается быть сохраненными в формате текстовых файлов.

    Параметры:
    model_save_path (str): Путь к директории, где сохранены модели.
    num_folds (int): Количество фолдов кросс-валидации, для которых были обучены модели.

    Возвращает:
    List[lgb.Booster]: Список загруженных моделей LightGBM.
    """
    loaded_models = []

    # Перебор и загрузка моделей для каждого фолда
    for i in range(1, num_folds + 1):
        # Формирование имени файла модели для каждого фолда
        model_filename = os.path.join(model_save_path, f'doblez_{i}.txt')
        # Проверка существования файла модели
        if os.path.exists(model_filename):
            # Загрузка модели из файла
            loaded_model = lgb.Booster(model_file=model_filename)
            loaded_models.append(loaded_model)
            print(f"Модель для фолда {i} загружена из {model_filename}")  # Перевод сообщения
        else:
            print(f"Файл модели {model_filename} не найден.")  # Перевод сообщения

    # Загрузка финальной модели (если она существует)
    final_model_filename = os.path.join(model_save_path, 'doblez-conjunto.txt')
    if os.path.exists(final_model_filename):
        final_model = lgb.Booster(model_file=final_model_filename)
        loaded_models.append(final_model)
        print(f"Финальная модель загружена из {final_model_filename}")  # Перевод сообщения
    else:
        print(f"Файл финальной модели {final_model_filename} не найден.")  # Перевод сообщения
    
    return loaded_models

# список папок, из которых нужно загрузить модели
folders = [
#     '/kaggle/input/lightgbm-models/modelitos_para_despues',
    '/kaggle/input/ensemble-of-models/results/modelitos_para_despues',
    '/kaggle/input/ensemble-of-models/results (1)/modelitos_para_despues',
    '/kaggle/input/ensemble-of-models/results (2)/modelitos_para_despues',
    '/kaggle/input/ensemble-of-models/results (3)/modelitos_para_despues',
     '/kaggle/input/ensemble-of-models/results (4)/modelitos_para_despues',
     '/kaggle/input/ensemble-of-models/results (5)/modelitos_para_despues',
    '/kaggle/input/ensemble-of-models/results (6)/modelitos_para_despues',
    '/kaggle/input/ensemble-of-models/results (7)/modelitos_para_despues',
]
num_folds = 5  # Количество фолдов для каждой модели
all_loaded_models = []  # Список для хранения всех загруженных моделей

# Перебор всех директорий и загрузка моделей из каждой из них
for folder in folders:
    all_loaded_models.extend(load_models_from_folder(folder))


### Submit

svm  модели которые были обучены локально 

In [ ]:
models = []
for i in range(1, 7):
    model = joblib.load(f"/kaggle/input/svm-kagle/svr_model_fold_{i}.joblib")
    models.append(model)

In [ ]:
models

In [ ]:
import numpy as np
import pandas as pd
import time
import optiver2023

def zero_sum(prices, volumes):
    """
    Корректирует предсказания так, чтобы их сумма была равна нулю.

    Параметры:
    prices (np.array): Массив предсказаний.
    volumes (np.array): Массив объемов, соответствующих предсказаниям.

    Возвращает:
    np.array: Корректированные предсказания.
    """
    std_error = np.sqrt(volumes)
    step = np.sum(prices) / np.sum(std_error)
    out = prices - std_error * step
    return out

if is_infer:
    # Инициализация среды Optiver и итератора тестовых данных
    env = optiver2023.make_env()
    iter_test = env.iter_test()
    counter = 0
    y_min, y_max = -64, 64  # Ограничения для предсказаний
    qps, predictions = [], []
    cache = pd.DataFrame()

    # Веса для моделей каждого фолда
    model_weights_lgbm = [1 / len(all_loaded_models)] * len(all_loaded_models)
    model_weights = [1 / len(models)] * len(models)
    
    # Обработка тестовых данных
    for (test, revealed_targets, sample_prediction) in iter_test:
        try:
            test = test.drop('currently_scored', axis=1)
            now_time = time.time()
            cache = pd.concat([cache, test], ignore_index=True, axis=0)
            if counter > 0:
                cache = cache.groupby(['stock_id']).tail(21).sort_values(by=['date_id', 'seconds_in_bucket', 'stock_id']).reset_index(drop=True)
            feat = generate_all_features(cache)[-len(test):]
            pred = np.zeros(len(test))

            # Предсказания с помощью поддерживающих моделей
            for model, weight in zip(all_loaded_models, model_weights_lgbm):
                pred += weight * model.predict(feat)

            # Генерация предсказаний для каждой модели и расчет взвешенного среднего
            svr_predictions = np.zeros(len(test))
            for model, weight in zip(models, model_weights):
                svr_predictions += weight * model.predict(feat.fillna(0.0))
                
            svr_predictions = [pred[i] if np.isnan(x) else x for i, x in enumerate(svr_predictions)]
            svr_predictions = (len(models) * svr_predictions + len(all_loaded_models) * pred) / (len(models) + len(all_loaded_models))
            svr_predictions = zero_sum(svr_predictions, test['bid_size'] + test['ask_size'])
            clipped_predictions = np.clip(svr_predictions, y_min, y_max)

            # Отправка предсказаний
            sample_prediction['target'] = clipped_predictions
            env.predict(sample_prediction)
        except:
            # В случае ошибки отправляем нулевое предсказание
            sample_prediction['target'] = 0.0
            env.predict(sample_prediction)
        counter += 1
        qps.append(time.time() - now_time)
        if counter % 10 == 0:
            print(counter, 'qps:', np.mean(qps))

    # Оценка времени выполнения кода
    time_cost = 1.146 * np.mean(qps)
    print(f"Код займет примерно {np.round(time_cost, 4)} часов для рассуждения")
